In [1]:
import cv2
import h5py
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import shutil

from tqdm import tqdm

In [2]:
from tactile_learning.datasets import dump_video_to_images, get_closest_id
from tactile_learning.utils.visualization import plot_tactile_sensor

In [3]:
import decimal

# Get the metadata of images and tactile information
def get_desired_indices(root, fps, view_num=0): # frames per second from the video to receive
    image_metadata_path = os.path.join(root, f'cam_{view_num}_rgb_video.metadata')
    tactile_info_path = os.path.join(root, 'touch_sensor_values.h5')

    with open(image_metadata_path, 'rb') as f:
        image_metadata = pickle.load(f)
        image_timestamps_array = np.asarray(image_metadata['timestamps'])
        image_timestamps = np.asarray(image_metadata['timestamps']) / 1000.
    with h5py.File(tactile_info_path, 'r') as f:
        tactile_timestamps = f['timestamps'][()]

    image_id, tactile_id = 0, 0
    curr_timestamp = tactile_timestamps[0] # These timestamps are in seconds
    image_id = get_closest_id(image_id, curr_timestamp, image_timestamps)

    tactile_indices, image_indices = [], []
    tactile_indices.append(tactile_id)
    image_indices.append(image_id)

    frame_period = 1. / fps
    while(True):
        curr_timestamp += frame_period
        tactile_id = get_closest_id(tactile_id, curr_timestamp, tactile_timestamps)
        image_id = get_closest_id(image_id, curr_timestamp, image_timestamps)

        if curr_timestamp > tactile_timestamps[tactile_id] and curr_timestamp > image_timestamps[image_id]:
            break

        tactile_indices.append(tactile_id)
        image_indices.append(image_id)

    assert len(tactile_indices) == len(image_indices)
    return tactile_indices, image_indices


In [4]:

def dump_states(root, tactile_indices, image_indices, view_num=0):
    # Make directory to dump the visualization
    pbar = tqdm(total=len(tactile_indices))

    with h5py.File(os.path.join(root, 'touch_sensor_values.h5'), 'r') as f:
        all_tactile_values = f['sensor_values'][()]

    viz_dir = os.path.join(root, 'visualization')
    os.makedirs(viz_dir, exist_ok=True)
    
    video_path = os.path.join(root, f'cam_{view_num}_rgb_video.avi')
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_id = 0
    for i in range(len(tactile_indices)):
        tactile_id, image_id = tactile_indices[i], image_indices[i]
        while frame_id != image_id and success:
            # Find the frame that is equal to image_id
            success, image = vidcap.read()
            frame_id += 1
        dump_demo_state(
            frame_id = i,
            viz_dir = viz_dir,
            tactile_values = all_tactile_values[tactile_id,:,:,:],
            camera_img = image
        )

        pbar.update(1)

    pbar.close()

def dump_demo_state(frame_id, viz_dir, tactile_values, camera_img):
    # tactile_values: (15,16,3)
    fig, axs = plt.subplots(figsize=(20,20), nrows=4, ncols=4)
    for row_id in range(4):
        for column_id in range(4):
            if row_id + column_id > 0: # The top left axis should stay empty
                plot_tactile_sensor(
                    ax = axs[column_id][row_id],
                    sensor_values = tactile_values[row_id*4 + column_id-1],
                    title = f'Sensor {row_id*4+column_id-1}'
                )
    fig.suptitle('Tactile State')
    fig.savefig(os.path.join(viz_dir, 'Tactile State.png'))
    fig.clf()
    plt.close()

    tactile_img = cv2.imread(os.path.join(viz_dir, 'Tactile State.png'))
    height_scale = camera_img.shape[0] / tactile_img.shape[0]
    tactile_img = cv2.resize(
        tactile_img,
        (int(tactile_img.shape[1] * height_scale),
         int(tactile_img.shape[0] * height_scale))
    )
    total_img = cv2.hconcat([camera_img, tactile_img])

    img_name = 'state_{}.png'.format(str(frame_id).zfill(3))
    cv2.imwrite(os.path.join(viz_dir, img_name), total_img)



In [16]:
import glob
# roots = [
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/bowl_picking/eval/demonstration_7',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/bowl_picking/eval/demonstration_9',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/bowl_picking/eval/demonstration_15',
# ]
data_path = '/home/irmak/Workspace/Holo-Bot/extracted_data/plier_picking'
# data_path = '/data/tactile_learning/deployment_data/data/bowl_picking/demonstrations/image_generalization'
roots = glob.glob(f'{data_path}/demonstration_*')

retrieval_fps = 1
view_num = 0
for root in roots:
    # print('root: {}'.format(root))
    tactile_indices, image_indices = get_desired_indices(root=root, fps=retrieval_fps, view_num=view_num)
    print('tactile_indices: {}, image_indices: {}'.format(tactile_indices, image_indices))
    dump_states(root, tactile_indices, image_indices, view_num=view_num)


tactile_indices: [0, 100, 198, 297, 395, 492, 591, 689, 786, 885, 984, 1081, 1179, 1277, 1374, 1474, 1571, 1670, 1768, 1866, 1963, 2062, 2160, 2257, 2356, 2460, 2552, 2650], image_indices: [71, 101, 130, 159, 188, 218, 247, 277, 305, 335, 365, 393, 423, 452, 481, 510, 540, 570, 599, 628, 658, 687, 716, 746, 775, 805, 834, 864]


100%|██████████| 28/28 [00:31<00:00,  1.14s/it]


tactile_indices: [0, 97, 196, 294, 392, 491, 589, 686, 784, 882, 979, 1079, 1176, 1274, 1372, 1471, 1568, 1667, 1765, 1864, 1962, 2060, 2158, 2257, 2356, 2451, 2550, 2648, 2746], image_indices: [69, 98, 128, 158, 187, 217, 246, 276, 306, 335, 365, 394, 423, 452, 481, 510, 539, 568, 597, 627, 656, 685, 714, 743, 772, 802, 832, 862, 892]


100%|██████████| 29/29 [00:33<00:00,  1.17s/it]


tactile_indices: [0, 99, 199, 298, 396, 495, 592, 691, 789, 887, 986, 1087, 1181, 1280, 1379, 1478, 1577, 1674], image_indices: [70, 100, 129, 158, 187, 217, 246, 275, 304, 333, 362, 392, 422, 451, 481, 510, 540, 569]


...

100%|██████████| 24/24 [00:27<00:00,  1.13s/it]


tactile_indices: [0, 101, 199, 298, 395, 494, 592, 690, 788, 887, 985, 1083, 1182, 1280, 1379, 1476, 1578, 1671, 1771, 1867, 1964, 2063, 2161, 2261, 2358, 2457, 2554, 2652, 2751, 2850, 2948, 3046, 3144, 3243, 3340, 3438, 3537, 3635, 3733, 3831, 3929, 4027, 4126, 4224], image_indices: [71, 99, 129, 159, 188, 217, 246, 276, 305, 334, 364, 394, 424, 453, 483, 512, 541, 571, 601, 630, 660, 689, 719, 748, 778, 808, 837, 866, 895, 925, 954, 982, 1012, 1042, 1071, 1101, 1131, 1160, 1189, 1219, 1249, 1278, 1307, 1334]


100%|██████████| 44/44 [00:50<00:00,  1.16s/it]


tactile_indices: [0, 100, 200, 299, 397, 496, 593, 692, 790, 889, 987, 1084, 1183, 1281, 1379, 1477, 1575, 1673, 1771, 1869, 1969, 2066, 2164, 2262, 2365, 2459, 2558, 2656, 2753, 2852], image_indices: [69, 99, 128, 157, 186, 215, 244, 274, 303, 333, 363, 392, 422, 452, 481, 510, 539, 569, 598, 627, 656, 685, 715, 744, 774, 803, 832, 862, 891, 921]


100%|██████████| 30/30 [00:35<00:00,  1.17s/it]


In [17]:
print(roots)

['/home/irmak/Workspace/Holo-Bot/extracted_data/plier_picking/demonstration_15', '/home/irmak/Workspace/Holo-Bot/extracted_data/plier_picking/demonstration_10', '/home/irmak/Workspace/Holo-Bot/extracted_data/plier_picking/demonstration_3', '/home/irmak/Workspace/Holo-Bot/extracted_data/plier_picking/demonstration_20', '/home/irmak/Workspace/Holo-Bot/extracted_data/plier_picking/demonstration_25', '/home/irmak/Workspace/Holo-Bot/extracted_data/plier_picking/demonstration_16', '/home/irmak/Workspace/Holo-Bot/extracted_data/plier_picking/demonstration_27']


In [18]:
# Turn the images to a video and delete the directory
video_fps = 10
for root in roots:
    print('dumping video in root: {}'.format(root))
    video_path = os.path.join(root, f'visualization_{view_num}.mp4')
    if os.path.exists(video_path):
        os.remove(video_path)
    viz_dir = os.path.join(root, 'visualization')
    os.system('ffmpeg -r {} -i {}/%*.png -vf scale=2000x720,setsar=1:1 {}'.format(
        video_fps, # fps
        viz_dir,
        video_path
    ))


ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 10.4.0 (conda-forge gcc 10.4.0-18)
  configuration: --prefix=/home/irmak/miniconda3/envs/tactile_learning --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-pthreads --enable-vaapi --enable-gpl --enable-libx264 --enable-libx265 --enable-libaom --enable-libsvtav1 --enable-libxml2 --enable-pic --enable-shared --disable-

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/plier_picking/demonstration_15


[libx264 @ 0x56159212e140] using SAR=1/1
[libx264 @ 0x56159212e140] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x56159212e140] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x56159212e140] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=10 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/plier_picking/demonstrat

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/plier_picking/demonstration_10


[libx264 @ 0x561102a0e1c0] using SAR=1/1
[libx264 @ 0x561102a0e1c0] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x561102a0e1c0] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x561102a0e1c0] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=10 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/plier_picking/demonstrat

...

[libx264 @ 0x561362aba3c0] using SAR=1/1
[libx264 @ 0x561362aba3c0] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x561362aba3c0] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x561362aba3c0] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=10 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/plier_picking/demonstrat

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/plier_picking/demonstration_16


[libx264 @ 0x558e95bbfec0] using SAR=1/1
[libx264 @ 0x558e95bbfec0] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x558e95bbfec0] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x558e95bbfec0] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=10 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/plier_picking/demonstrat

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/plier_picking/demonstration_27


[libx264 @ 0x555d95ad5240] using SAR=1/1
[libx264 @ 0x555d95ad5240] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x555d95ad5240] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x555d95ad5240] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=10 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/plier_picking/demonstrat

In [19]:
# Remove the img directory

for root in roots:
    viz_dir = os.path.join(root, 'visualization')
    shutil.rmtree(viz_dir)
